In [1]:
import vtk
import numpy as np
import pandas as pd
import time
import re



In [2]:
# 读取 STL 文件
reader = vtk.vtkSTLReader()
reader.SetFileName("medtronic_chicken_foot.STL")
reader.Update()

model = reader.GetOutput()

# 创建 mapper 和 actor
mapper = vtk.vtkPolyDataMapper()
mapper.SetInputData(model)

actor = vtk.vtkActor()
actor.SetMapper(mapper)

# 创建 render window, renderer 和 interactor
renderer = vtk.vtkRenderer()
render_window = vtk.vtkRenderWindow()
render_window.AddRenderer(renderer)

# 设置渲染器大小
render_window.SetSize(2000, 1600)

# 创建交互器
render_window_interactor = vtk.vtkRenderWindowInteractor()
render_window_interactor.SetRenderWindow(render_window)

# 将 actor 添加到 renderer 中
renderer.AddActor(actor)
# renderer.SetBackground(0.1, 0.1, 0.1)

# 创建坐标轴
axes = vtk.vtkAxesActor()
axes.SetTotalLength(200, 200, 200)  # 设置坐标轴的长度
axes.SetAxisLabels(True)

# 设置坐标轴标签的位置
axes.GetXAxisCaptionActor2D().GetTextActor().GetPositionCoordinate().SetCoordinateSystemToNormalizedDisplay()
axes.GetYAxisCaptionActor2D().GetTextActor().GetPositionCoordinate().SetCoordinateSystemToNormalizedDisplay()
axes.GetZAxisCaptionActor2D().GetTextActor().GetPositionCoordinate().SetCoordinateSystemToNormalizedDisplay()

# 设置坐标轴标签
axes.GetXAxisCaptionActor2D().GetTextActor().GetProperty().SetColor(1, 0, 0)  # X轴为红色
axes.GetYAxisCaptionActor2D().GetTextActor().GetProperty().SetColor(0, 1, 0)  # Y轴为绿色
axes.GetZAxisCaptionActor2D().GetTextActor().GetProperty().SetColor(0, 0, 1)  # Z轴为蓝色

# 将坐标轴添加到 renderer
renderer.AddActor(axes)

# 设定摄像机视角，固定在 (0, 0, -3000) 位置
renderer.GetActiveCamera().SetPosition(0, 0, -3000)  # 摄像机位置
renderer.GetActiveCamera().SetFocalPoint(0, 0, 0)  # 摄像机焦点
renderer.GetActiveCamera().SetViewUp(-1, 0, 0)  # 设置“上”方向
renderer.GetActiveCamera().SetClippingRange(1, 3000)  # 设置裁剪范围


In [3]:
# 读取 tracking 数据
recording = pd.read_csv('recording.csv', header=None)
frame_idx = 0  # 记录当前帧索引

In [4]:
# 定义定时器回调函数
def update_pose(obj, event):
    global frame_idx

    row = recording.iloc[frame_idx]
    tracking = row[3]  # 提取 tracking 数据
    numbers = [float(x) if x != "nan" else np.nan for x in re.findall(r"nan|[-+]?\d*\.\d+e[+-]?\d+|[-+]?\d*\.\d+|\d+", tracking)]
    tracking_matrix = np.array(numbers).reshape(4, 4)

    # VTK 变换
    transform = vtk.vtkTransform()
    transform.SetMatrix(tracking_matrix.flatten())  # 设置变换矩阵
    actor.SetUserTransform(transform)  # 应用变换

    render_window.Render()  # 重新渲染
    frame_idx += 1  # 更新帧索引

In [5]:
def close_window(obj, event):
    render_window_interactor.GetRenderWindow().Finalize()  # Properly release resources
    render_window_interactor.TerminateApp()  # Exit VTK loop

In [6]:
# 设定定时器，每 10ms 触发一次
render_window_interactor.AddObserver("TimerEvent", update_pose)
render_window_interactor.CreateRepeatingTimer(1)  # 10ms
render_window_interactor.AddObserver("KeyPressEvent", close_window)

# 启动交互模式（不会阻塞后续代码）
render_window.Render()
render_window_interactor.Initialize()
render_window_interactor.Start()


2025-02-14 16:19:06.343 python[76320:12992461] +[IMKClient subclass]: chose IMKClient_Modern
2025-02-14 16:19:06.343 python[76320:12992461] +[IMKInputSession subclass]: chose IMKInputSession_Modern


: 